# 옵션 이론가

BS([underlyingPrice, strikePrice, interestRate, daysToExpiration],volatility=x)

underlyingPrice : 현재 거래되는 주가

strikePrice : 옵션 행사가

interestRate : 무위험 이자율 (선물가격을 underlyingPrice 로 사용할 경우는 이미 future price 에 이자율이 포함되어 있으므로 0 으로 함)

daysToExpiry : 잔존일수

volatility : 옵션의 내재변동성

In [1]:
import mibian
from tabulate import tabulate

In [2]:
stock_price = 340.3
strike_price = 350
interest_rate = 0
days_to_expiry = 29
volatility = 30.0

c = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], volatility=volatility)

In [3]:
c.callPrice

7.426905031477048

In [4]:
c.putPrice

17.12690503147701

# Greek Calculator

In [5]:
greeks = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], volatility=volatility)

# Greeks for call option

In [6]:
print(tabulate(
    [
        ['Delta', greeks.callDelta],
        ['Gamma', greeks.gamma],
        ['Vega', greeks.vega],
        ['Theta', greeks.callTheta],
        ['Rho', greeks.callRho]
    ],
    headers=['Greeks', 'Value']))

print("")
print('Call price is:', greeks.callPrice)

Greeks         Value
--------  ----------
Delta      0.385875
Gamma      0.0132923
Vega       0.366903
Theta     -0.189778
Rho        0.0984302

Call price is: 7.426905031477048


# Greeks for put option

In [7]:
print(tabulate(
    [
        ['Delta', greeks.putDelta],
        ['Gamma', greeks.gamma],
        ['Vega', greeks.vega],
        ['Theta', greeks.putTheta],
        ['Rho', greeks.putRho]
    ],
    headers=['Greeks', 'Value']))

print("")
print('Put price is:', greeks.putPrice)

Greeks         Value
--------  ----------
Delta     -0.614125
Gamma      0.0132923
Vega       0.366903
Theta     -0.189778
Rho       -0.179652

Put price is: 17.12690503147701


# Delta 와 Gamma 를 이용한 Call Price 계산

**Call Option Price = Initial Option Price + Delta * (change in the stock price) + 0.5 * Gamma * (change in the stock price)^2**

In [19]:
initial_stock_price = 100
strike_price = 100
interest_rate = 0
days_to_expiry = 40
volatility = 38.09

# initial call option price
greeks_stock_price_100 = mibian.BS([initial_stock_price, strike_price, interest_rate, days_to_expiry], volatility=volatility)

print(tabulate(
    [
        ['주가', initial_stock_price],
        ['콜옵션 가격', greeks_stock_price_100.callPrice],
        ['Delta', greeks_stock_price_100.callDelta],
        ['Gamma', greeks_stock_price_100.gamma]
    ],
    tablefmt="grid"))

+-------------+-------------+
| 주가        | 100         |
+-------------+-------------+
| 콜옵션 가격 |   5.02709   |
+-------------+-------------+
| Delta       |   0.525135  |
+-------------+-------------+
| Gamma       |   0.0315757 |
+-------------+-------------+


### 위의 주식이 101 로 가격이 바뀔때 Delta 와 Gamm 를 이용한 call price 계산

In [20]:
# change in stock price from 100 to 101
changed_stock_price = 101
stock_price_change = changed_stock_price - initial_stock_price

call_price_at_101 = greeks_stock_price_100.callPrice + greeks_stock_price_100.callDelta * stock_price_change \
                             + 0.5 * greeks_stock_price_100.gamma * stock_price_change**2
    
print(call_price_at_101)

5.56801680203463


In [23]:
# BS model 로 검증
mibian.BS([changed_stock_price, strike_price, interest_rate, days_to_expiry], volatility=volatility).callPrice

5.567930205927425

## Vega 를 이용한 call price 계산

**Price of Call option = Initial Option price + Vega * (Change in Volatility) x 100**

### 내재변동성이 30% 인 경우의 Vega 계산

In [30]:
stock_price = 100
strike_price = 100
interest_rate = 0
days_to_expiry = 40
volatility = 30

greeks_stock_price_100 = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], volatility=volatility)

print(tabulate(
    [
        ['주가', stock_price],
        ['콜옵션 가격', greeks_stock_price_100.callPrice],
        ['Vega', greeks_stock_price_100.vega]
    ],
    tablefmt="grid"))

+-------------+------------+
| 주가        | 100        |
+-------------+------------+
| 콜옵션 가격 |   3.96038  |
+-------------+------------+
| Vega        |   0.131904 |
+-------------+------------+


### 위 옵션의 내재변동성이 31% 가 된 경우의 call price 계산

In [31]:
call_price_at_31 = greeks_stock_price_100.callPrice + greeks_stock_price_100.vega * (0.31 - 0.3) * 100

print(call_price_at_31)

4.092280217433506


In [32]:
# BS model 로 검증
mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], volatility=31).callPrice

4.092274736640356

In [33]:
import numpy as np

In [35]:
np.exp(-0.06)*900 + 60 - 900

7.588080225823887